In [1]:
import sys
src_dir = '/Users/palmerio/Science_projects/GRB_population_model/grbpop/'
sys.path.insert(0, src_dir)

# GRB pop module imports
from GRB_population import GRBPopulation
import physics as ph
import stats as st
import io_grb_pop as io
import miscellaneous as msc
import plotting_functions as pf
from cosmology import init_cosmology
from ECLAIRs import init_ECLAIRs

# General imports
import yaml
import logging
import corner
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

# specific imports
from scipy.stats import ks_2samp
from scipy.stats import mstats
from itertools import product

log = logging.getLogger(__name__)
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG,
                    format='%(asctime)s.%(msecs)03d [%(levelname)s] %(message)s',
                    datefmt='%H:%M:%S')
logging.getLogger('matplotlib').setLevel(logging.WARNING)

plt.style.use('paper')

In [2]:
paths_to_dir, paths_to_files = io.generate_paths()

15:10:35.506 [DEBUG] Root directory is /Users/palmerio/Science_projects/GRB_population_model
15:10:35.507 [DEBUG] Directory paths :
    'root'    : /Users/palmerio/Science_projects/GRB_population_model
    'obs'     : /Users/palmerio/Science_projects/GRB_population_model/observational_constraints
    'init'    : /Users/palmerio/Science_projects/GRB_population_model/init
    'output'  : /Users/palmerio/Science_projects/GRB_population_model/model_outputs
    'data'    : /Users/palmerio/Science_projects/GRB_population_model/data
    'cosmo'   : /Users/palmerio/Science_projects/GRB_population_model/data/cosmology
    'ECLAIRs' : /Users/palmerio/Science_projects/GRB_population_model/data/ECLAIRs
15:10:35.507 [DEBUG] File paths :
    'config'          : /Users/palmerio/Science_projects/GRB_population_model/init/config.yml
    'param'           : /Users/palmerio/Science_projects/GRB_population_model/init/parameters.yml
    'instrum'         : /Users/palmerio/Science_projects/GRB_population_mo

In [ ]:
# Generate the populations for the best model in each scenario
np.random.seed(1)
model_names = ['kF_LN_nSFR',
               'k0_LN_nF', 
               'k05_LN_nF', 
               'k1_LN_nF', 
               'k2_LN_nF', 
               'kF_A_nSFR',
               'k0_A_nF', 
               'k05_A_nF',
               'k05_A_nF_2',
               'k1_A_nF', 
               'k1_A_nF_2', 
               'k2_A_nF']

model_names_ks = ['k0_LN_nF', 'k2_LN_nF', 'k0_A_nF', 'k2_A_nF']
# colors = ['#225ea8', '#41b6c4', '#ae017e', '#f768a1']
colors = ['#225ea8', '#41b6c4', '#d94801', '#fd8d3c']

parameters = {}
pops = {}
for mod in model_names:
    print(f'Calculating {mod}... ')
    with open(paths_to_files['param'].parent / f'parameters_{mod}.yml', 'r') as f:
        parameters[mod] = yaml.safe_load(f)

    dfs[mod] = GRBPopulation(Nb_GRBs, output_dir=paths_to_dir['output'])
    dfs[mod].draw_GRB_properties(cosmo=cosmo, params=parameters[mod], run_mode=None)
    dfs[mod].calculate_quantities(instruments=incl_instruments, samples=incl_samples, ECLAIRs_prop=ECLAIRs_prop)
    dfs[mod].create_mock_constraints(constraints=incl_constraints)
    dfs[mod].compare_to_observational_constraints(constraints=incl_constraints)
    dfs[mod].normalize_to_Stern()
